In [ ]:
import numpy as np
import numpy.random as random
import pandas as pd

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl


FONT_SIZE = 12
AXES_SIZE = FONT_SIZE * 1.5
TICK_SIZE = FONT_SIZE * 1.25
LEGEND_SIZE = FONT_SIZE

my_rcParams = {
    "axes.spines.top": False,
    "axes.spines.right": False,
    "lines.linewidth": 5,
    "font.size": FONT_SIZE,
    "font.family": "Helvetica",
    "axes.titlesize": AXES_SIZE,
    "axes.labelsize": AXES_SIZE,
    "xtick.labelsize": TICK_SIZE,
    "ytick.labelsize": TICK_SIZE,
    "legend.fontsize": LEGEND_SIZE,
    "xtick.major.pad": FONT_SIZE / 2,
    "ytick.major.pad": FONT_SIZE / 2,
}

for k, v in my_rcParams.items():
    mpl.rcParams[k] = v

In [ ]:
variation_name = ["WW", "SR"]
variation_colors = ["#30BCED", "#A6FFA1"]
vc_map = dict(zip(variation_name, variation_colors))

CLICK_RATE = np.array([0.1, 0.15])
visitors_per_day= np.array([500])
assignment_p = np.ones_like(CLICK_RATE) / len(CLICK_RATE)

# Sample data 
def sample_clicks(visitors, click_rate, assignment_p):
    assigned_variations = random.multinomial(visitors, pvals=assignment_p)
    observed_clicks = [random.binomial(n_assigned, click_rate[variation]) for variation, n_assigned in enumerate(assigned_variations)] 
    return assigned_variations, observed_clicks

def make_data(visitors_per_day, click_rate, assignment_p):
    dfs = []
    for day, visits in enumerate(visitors_per_day):
        impressions, clicks = sample_clicks(visits, click_rate, assignment_p)
        df_today = pd.DataFrame({
            "variation": variation_name,
            "impressions": impressions,
            "clicks": clicks,
            "day": day
            })
        dfs.append(df_today)
    return pd.concat(dfs)
    
data = make_data(visitors_per_day, CLICK_RATE, assignment_p)
data.to_csv("../ad_clicks.csv", index=False)

In [ ]:
# Plot posterior intervals of click rate through time
from scipy.stats import beta 

posterior_samples = 5_000

posterior_click_rates_dict = {}

# Sum by variation
data_agg = data.groupby("variation").agg({"clicks": "sum", "impressions": "sum"}).reset_index()
def _compute_posterior(clicks, impressions, a=1, b=1):
    return beta(a + clicks, b + impressions - clicks)

priors = {v: _compute_posterior(np.zeros_like(g["clicks"]), np.zeros_like(g["impressions"])) for v, g in data_agg.groupby("variation")}
posteriors = {v: _compute_posterior(g["clicks"], g["impressions"]) for v, g in data_agg.groupby("variation")}


In [ ]:
fig = plt.figure(figsize=(10., 5.), constrained_layout=True)
spec = fig.add_gridspec(ncols=1, nrows=len(variation_name))
for i, v in enumerate(variation_name):
    ax = fig.add_subplot(spec[i])
    ax.hist(priors[v].rvs(size=posterior_samples), color=vc_map[v], ec="k", alpha=0.2, label=f"Prior ({v})", bins=20,)
    ax.hist(posteriors[v].rvs(size=posterior_samples), color=vc_map[v], ec="k", label=f"Posterior ({v})", bins=20)
    ax.set_xlabel("Click through rate")
    ax.set_ylabel("Density")
    ax.legend()

fig.savefig("../figures/prior-posterior.png")

In [ ]:
fig = plt.figure(figsize=(10., 6), constrained_layout=True)
spec = fig.add_gridspec(ncols=1, nrows=3)

alphasbetas = [1, 25, 500]
alphasbetas_colors = ["lightblue", "slateblue", "darkblue"]

def get_prior_post(alpha, beta):
    prior = {v: _compute_posterior(np.zeros_like(g["clicks"]), np.zeros_like(g["impressions"]), a=alpha, b=beta) for v, g in data_agg.groupby("variation")}
    posteriors = {v: _compute_posterior(g["clicks"], g["impressions"], a=alpha, b=beta) for v, g in data_agg.groupby("variation")}
    return prior, posteriors

for i, ab in enumerate(alphasbetas):
    ax = fig.add_subplot(spec[i])
    priors_, posteriors_ = get_prior_post(alpha=ab, beta=ab)
    ax.hist(priors_[v].rvs(size=posterior_samples), 
            alpha=0.2, color=alphasbetas_colors[i], ec="k", label=r"Prior($\alpha=\beta=$" + str(ab) + ")", bins=20)
    ax.hist(posteriors_[v].rvs(size=posterior_samples),
             color=alphasbetas_colors[i], ec="k", label="Posterior", bins=20)
    ax.axvline(x=CLICK_RATE[0], color="k", label="True value")
    ax.set_xlabel("Click through rate")
    ax.set_ylabel("Density")
    ax.set_xlim((0,1))
    ax.legend()

fig.savefig("../figures/changing-prior-posterior.png")

In [ ]:
# Plot posteriors
fig = plt.figure(figsize=(10., 6.), constrained_layout=True)
spec = fig.add_gridspec(ncols=1, nrows=2)
ax = fig.add_subplot(spec[0])
ax_greater_than = fig.add_subplot(spec[1])
for v, dist in posteriors.items():
    ax.hist(dist.rvs(size=posterior_samples),  color=vc_map[v], ec="k", alpha=0.5, label=v, bins=20, density=True)
ax.set_xlabel("Click through rate")
ax.set_ylabel("Density")
ax.legend()

# Plot differences
def compute_lift(posteriors, p1, p0, size=1000):
    if p1 == p0:
        return np.zeros(size)
    else:
        return posteriors[p1].rvs(size=size) - posteriors[p0].rvs(size=size) 

lift = compute_lift(posteriors, variation_name[-1], variation_name[0], size=posterior_samples)
ax_greater_than.hist(lift, 
                     color="purple",
                     ec="k",
                     alpha=0.5,
                     label=f"Lift of {variation_name[-1]} over {variation_name[0]}",
                     bins=20,
                     density=True)
ax_greater_than.axvline(x=0.0, color="k", linestyle="--", linewidth=2.5)
ax_greater_than.set_xlabel("Lift")
ax_greater_than.set_ylabel("Density")
ax_greater_than.legend()

fig.savefig("../figures/posterior-lift.png")

In [ ]:
# How do we make a conclusion with this data?
def compute_expected_loss(posteriors, size=1000):
    # Basically want to compute lift for all things and take maximum
    loss = {}
    expected_loss = {}
    for p in posteriors.keys():
        lifts = np.array([compute_lift(posteriors, p, ps) for ps in posteriors.keys()])
        loss[p] = (-lifts).max(axis=0)
        expected_loss[p] = loss[p].mean()
    return loss, expected_loss

def make_decision(thres_caring, expected_losses):
    below_thres = {k: v < thres_caring for k,v in expected_losses.items()}
    for k, v in below_thres.items():
        if v:
            print(f"{k} is acceptable")

thres_caring = 0.02 # 
make_decision(thres_caring, expected_losses)# This can work for an arbritary loss function, say if there is a cost for swtichign


#expected_losses = compute_expected_loss(posteriors)
#for p, expected_loss in expected_losses.items():
#    ax_greater_than.axvline(x=expected_loss, color=vc_map[p])


#ax_greater_than.axvline(x=thres_caring, color="k", linewidth=2.5, label="Threshold of caring")
#ax_greater_than.legend()

In [ ]:
fig = plt.figure(figsize=(10., 6.), constrained_layout=True)
spec = fig.add_gridspec(ncols=1, nrows=2)

loss, expected_loss= compute_expected_loss(posteriors)
for i, v in enumerate(variation_name):
    ax = fig.add_subplot(spec[i], sharex= None if i==0 else ax)
    ax.hist(loss[v], color=vc_map[v], ec="k", alpha=0.5, label=v, bins=20)
    ax.axvline(x=expected_loss[v], color="k", label= "Expected loss")
    ax.set_xlabel("Loss")
    ax.legend()

fig.savefig("../figures/posterior-loss.png")

In [ ]:
thres_caring = 0.01 # 

fig = plt.figure(figsize=(6, 4.), constrained_layout=True)
spec = fig.add_gridspec(ncols=1, nrows=1)
ax = fig.add_subplot(spec[0])

loss, expected_loss= compute_expected_loss(posteriors)

ax.axhline(y=thres_caring, color="k", linewidth=5.5, label="Threshold of caring")
ax.bar(expected_losses.keys(), expected_losses.values(), color=[vc_map[v] for v in expected_losses.keys()], ec="k")
ax.set_ylabel("Expected Loss")
ax.legend()

fig.savefig("../figures/expected-loss.png")

In [ ]:
# Plot posterior intervals of click rate through time
from scipy.stats import beta 

posterior_samples = 5000

fig = plt.figure(figsize=(12., 6.), constrained_layout=True)
spec = fig.add_gridspec(ncols=2, nrows=2, width_ratios=[1.0, 0.5])
ax = fig.add_subplot(spec[0, 0])
ax_greater_than = fig.add_subplot(spec[1, 0])
ax_expected_loss = fig.add_subplot(spec[:, 1])
posterior_click_rates_dict = {}

# Sum by variation
data_agg = data.groupby("variation").agg({"clicks": "sum", "impressions": "sum"}).reset_index()
def _compute_posterior(clicks, impressions, a=1, b=1):
    return beta(a + clicks, b + impressions - clicks)

posteriors = {v: _compute_posterior(g["clicks"], g["impressions"]) for v, g in data_agg.groupby("variation")}

# Plot posteriors
for v, dist in posteriors.items():
    ax.hist(dist.rvs(size=posterior_samples),  color=vc_map[v], ec="k", alpha=0.5, label=v, bins=20, density=True)
ax.set_xlabel("Click through rate")
ax.set_ylabel("Density")
ax.legend()

# Plot differences
def compute_value(posteriors, p1, p0, size=1000):
    if p1 == p0:
        return np.zeros(size)
    else:
        return posteriors[p1].rvs(size=size) - posteriors[p0].rvs(size=size)

# Compute values
relative_to = variation_name[0]
for variation in variation_name[1:]:
    value = compute_value(posteriors, variation, relative_to, size=posterior_samples)
    ax_greater_than.hist(value, ec="k", color=vc_map[variation], alpha=0.5, label=f"Value of {variation} over {relative_to}", bins=20, density=True)

ax_greater_than.axvline(x=0.0, color="k", linestyle="--", linewidth=2.5)
ax_greater_than.set_xlabel("Lift")
ax_greater_than.set_ylabel("Density")

# Making a decision with this data
def compute_expected_loss(posteriors, size=1000):
    # Basically want to compute lift for all things and take maximum
    expected_loss = {}
    def compute_loss(posteriors, p1, p0, size=size):
        relative_lift = compute_value(posteriors, p1, p0)
        loss = (-relative_lift)
        return loss 
    
    for p in posteriors.keys():
        # Consider values if we try alternatives to p when p is true
        loss = np.array([compute_loss(posteriors, p, alt) for alt in posteriors.keys()])
        expected_loss[p] = loss.max(axis=0).mean() # Loss occurs when alternatives are better (max_value > 0)
    return expected_loss


def make_decision(thres_caring, expected_losses):
    below_thres = {k: v < thres_caring for k,v in expected_losses.items()}
    for k, v in below_thres.items():
        if v:
            print(f"{k} is acceptable")

thres_caring = 0.01 # 
make_decision(thres_caring, expected_losses)# This can work for an arbritary loss function, say if there is a cost for swtichign
ax_greater_than.legend()

ax_expected_loss.axhline(y=thres_caring, color="k", linewidth=5.5, label="Threshold of caring")
expected_losses = compute_expected_loss(posteriors)
ax_expected_loss.bar(expected_losses.keys(), expected_losses.values(), color=[vc_map[v] for v in expected_losses.keys()], label=f"Choosing {p}", ec="k")
ax_expected_loss.set_ylabel("Expected Loss")
print(expected_losses)

# If you use relative lift, you're losing by going A, so loss is A/B - 1, but gain by choosing would be B/A - 1

In [ ]:
# Repeat with weighted example for binary decision


I want figures that will allow me to look at sample size needed given power, alpha as a function of effect size?

The follow up here would be then leaveraging A/B test results for some other outcome.

Which will lead to actual benefit.

Say we have paid users verus unpaid users. How do we maximize profit of our choice ?

In [ ]:
class SimpleRateExperiment:
    def __init__(self, counts, totals, group, a_prior=1, b_prior=1, size=1_000):
        self.counts, self.totals = counts, totals
        self.group = group
        self.a_prior = a_prior
        self.b_prior = b_prior

        self.size = size
        self.posterior = {}
        self.samples = {}
        self.expected_losses = {}

    def _compute_posterior(self, data):
        for name, group in data.groupby(self.group):
            self.posterior[name] = beta(self.a_prior + group[self.counts], self.b_prior + group[self.totals] - group[self.counts])
    
    def _sample_posterior(self):
        for name, post in self.posterior.items():
            self.samples[name] = post.rvs(self.size)

    def _compute_lift(self, current, alternative):
        return self.samples[alternative] - self.samples[current]

    def decide(self, thres_caring=0.01):
        # Compute expected loss
        for current, post in self.posterior.items():
            lifts = np.asarray([self._compute_lift(current, alt) for alt in self.posterior.keys()]) 
            self.expected_losses[current] = -(-lifts).min(axis=0).mean()
        
        # Check to see which alternatives if any meet threshold of caring
        threshold_met = False
        for alt, expected_loss in self.expected_losses.items():
            if expected_loss < thres_caring:
                print(f"{alt} is acceptable")
                threshold_met = True
        if threshold_met:
            self.decision = min(expected_losses, key=expected_losses.get)
            print(f"We choose {self.decision}.")
        else:
            print("No alternatives met the threshold of caring.")
    
    def run_test(self, data, thres_caring=None):
        # Fit individual count models
        self._compute_posterior(data)
        self._sample_posterior()

        # Make decision
        if thres_caring is not None:
            self.decide(thres_caring)

In [ ]:
experiment = SimpleRateExperiment(counts="clicks", totals="impressions", group="variation")
experiment.run_test(data_agg, thres_caring=0.01)